In [ ]:
import pickle

import matplotlib.pyplot as plt
import healpy as hp
import numpy as np

from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator

In [ ]:
data = pickle.load(open("/home/laclavere/Documents/Thesis/qubic/qubic/scripts/MapMaking/src/CMM/parametric_d0_DB_test_parametric/maps/file_seed1_None_k9.pkl", "rb"))

In [ ]:
fwhm_in = data["fwhm_in"]
fwhm_out = data["fwhm_out"]
fwhm_rec = data["fwhm_rec"]

comp_maps_input = data["components"]
comp_maps_rec = data["components_i"]
center = data["center"]
seenpix = data["seenpix"]

In [ ]:
print(comp_maps_input.shape)

In [ ]:
print("FWHM in: ", fwhm_in)
print("FWHM out: ", fwhm_out)
print("FWHM rec: ", fwhm_rec)

In [ ]:
name_comp = ["CMB", "Dust"]
STOKES = ["I", "Q", "U"]
k = 0

comp_maps_input[:, ~seenpix, :] = hp.UNSEEN
comp_maps_rec[:, ~seenpix, :] = hp.UNSEEN

istk = 0
n = 2
sigma_input = np.std(comp_maps_input[:, seenpix, ], axis=0)
sigma_rec = np.std(comp_maps_rec[:, seenpix, ], axis=0)
sigma_res = np.std(comp_maps_rec[:, seenpix, ] - comp_maps_input[:, seenpix, ], axis=0)

plt.figure(figsize=(20, 10))
for icomp in range(2):
    hp.gnomview(comp_maps_input[icomp, :, istk], rot=center,
                        reso=15,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} input",  
                        cmap="jet",
                        min = - n * sigma_input,
                        max = n * sigma_input,
                        sub=(2, 3, k + 1))
    
    hp.gnomview(comp_maps_rec[icomp, :, istk], rot=center,
                        reso=15,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Output",  
                        cmap="jet",
                        min = - n * sigma_input,
                        max = n * sigma_input,
                        sub=(2, 3, k + 2))
    
    hp.gnomview((comp_maps_rec[icomp, :, istk] - comp_maps_input[icomp, :, istk]), rot=center,
                        reso=15,
                        notext=True,
                        title=f"{name_comp[icomp]} {STOKES[istk]} Residual",  
                        cmap="jet",
                        min = - n * sigma_input,
                        max = n * sigma_input,
                        sub=(2, 3, k + 3))
    k += 3

In [ ]:
import time

from pyoperators import IdentityOperator

time0 = time.time()
C_0 = HealpixConvolutionGaussianOperator(0)
print("Time for C_0: ", time.time() - time0)
timefwhm = time.time()
C_fwhm = HealpixConvolutionGaussianOperator(fwhm_rec[-1])
print("Time for C_fwhm: ", time.time() - timefwhm)
timeI = time.time()
C_I = IdentityOperator
print("Time for C_I: ", time.time() - timeI)

print(C_0 == C_I)
print(C_0)
print(C_I)
print(C_fwhm)

In [ ]:
print(fwhm_in[-1])